<a href="https://colab.research.google.com/github/Bentley97/NLU_First_assignment/blob/main/FirstAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FIRST ASSIGNMENT**

Student:
*   Name: Luca
*   Surname: Bentivoglio
*   Student number: 221246



Hoping it will be appriciated, I created a Colab notebook because I had some problems installing spaCy on my personal laptop.

Anyway, I put code and report for each function all together in the notebook. If you want to run it, you should be able to open it.

## **REQUIREMENTS**

If you open the notebook, is sufficient to run the first block of code to import spaCy, define a sentence for tests and load the dependency parsing model.
Then you can run each function with its test calls.

In [124]:
import spacy

test_sentence = "I saw a man with a telescope."

nlp = spacy.load('en_core_web_sm')

## **FUNCTION 1**  -  `extract_dependency_path(sentence)`
### Extract a path of dependency relations from the ROOT to a token.

This function parses the sentence given in input and extracts for each token the dependency path from the ROOT to the token itself.
This is accomplished, starting from the token, exploring backwards arcs in the dependency graph/tree thanks to the attribute `head` that gives the parent of the token, until it reaches the ROOT, and the attribute `dep_` that gives the dependency relation between the token and its parent.


Input:
* sentence ==> a string

Output:
* the list of paths

Each path is structured as follows:

['token1', '--dependencyRelation1->', 'token2', '--dependencyRelation2->', 'token3']

In [ ]:
def extract_dependency_path(sentence):
  doc = nlp(sentence)

  paths = []

  for token in doc:
    token_path = []

    while token != token.head:  #we can use also (token != token.sent.root)
      token_path.insert(0, token.text)
      token_path.insert(0, "--"+token.dep_+"->")      #another format:  token_path.insert(0, "goes with dep '"+token.dep_+"' in")
      token = token.head

    token_path.insert(0, token.text)
    token_path.insert(0, "--"+token.dep_+"->")        #another format:  token_path.insert(0, "goes with dep '"+token.dep_+"' in")
    token_path.insert(0, "ROOT")
    paths.append(token_path)
  
  return paths


In [ ]:
dep_paths = extract_dependency_path(test_sentence)

for dep_path in dep_paths:
  print("The dependency path for token '{}'\tis:\t{}".format(dep_path[-1],dep_path).expandtabs(15))

The dependency path for token 'I'            is:            ['ROOT', '--ROOT->', 'saw', '--nsubj->', 'I']
The dependency path for token 'saw'          is:            ['ROOT', '--ROOT->', 'saw']
The dependency path for token 'a'            is:            ['ROOT', '--ROOT->', 'saw', '--dobj->', 'man', '--det->', 'a']
The dependency path for token 'man'          is:            ['ROOT', '--ROOT->', 'saw', '--dobj->', 'man']
The dependency path for token 'with'         is:            ['ROOT', '--ROOT->', 'saw', '--dobj->', 'man', '--prep->', 'with']
The dependency path for token 'a'            is:            ['ROOT', '--ROOT->', 'saw', '--dobj->', 'man', '--prep->', 'with', '--pobj->', 'telescope', '--det->', 'a']
The dependency path for token 'telescope'    is:            ['ROOT', '--ROOT->', 'saw', '--dobj->', 'man', '--prep->', 'with', '--pobj->', 'telescope']
The dependency path for token '.'            is:            ['ROOT', '--ROOT->', 'saw', '--punct->', '.']


## **FUNCTION 2**  -  `extract_subtrees_of_dependents(sentence)`
### Extract subtree of a dependents given a token 

This function  parses the sentence in input and for each token extracts the subtree of its dependents ordered w.r.t. sentence order.

Premising that I think a subtree is not a tree without its root, I did not understand clearly what is the request, since returning simply the subtree attribute seeemed to me too trivial.
So I wrote two functions:
1. `extract_subtrees_of_dependents(sentence)`: this one extracts for each token its subtree (ordered w.r.t sentence order) removing the token itself, since in the description of the requirement there is "extract a subtree of its dependents", so I extracted only the dependents of the token ordered w.r.t sentence order;
1. `extract_subtrees(sentence)`: this one extracts the complete subtree of each token ordered w.r.t sentence order.

Input:
* sentence ==> a string

Output:
* a list of lists containing the token and the relative subtree  ([token, subtree])

The output element is structured as follows:

[
  [token1, [tokenA, tokenB, tokenC]],
  [token2, [tokenA, tokenB, tokenC]],
  [token3, [tokenA, tokenB, tokenC]]
]


In [125]:
def extract_subtrees_of_dependents(sentence):
  doc = nlp(sentence)

  result = []
  
  for token in doc:
    subtree_list = []
    for element in token.subtree:
      if element != token:
        subtree_list.append(element)
    result.append([token, subtree_list])

  return result  #returns a list of elements, each composed by the token and the list of its dependents in its subtree(without himself) ordered w.r.t. sentence order


def extract_subtrees(sentence):
  doc = nlp(sentence)

  result = []

  for token in doc:
    result.append([token, list(token.subtree)])

  return result


In [ ]:
subtrees = extract_subtrees_of_dependents(test_sentence)
print("Function: extract_subtrees_of_dependents")
for t,s in subtrees:
  print("Token: {}\t{}".format(t,s).expandtabs(20))

print("\n\n")

subtrees = extract_subtrees(test_sentence)
print("Function: extract_subtrees")
for t,s in subtrees:
  print("Token: {}\t{}".format(t,s).expandtabs(20))

Function: extract_subtrees_of_dependents
Token: I            []
Token: saw          [I, a, man, with, a, telescope, .]
Token: a            []
Token: man          [a, with, a, telescope]
Token: with         [a, telescope]
Token: a            []
Token: telescope    [a]
Token: .            []



Function: extract_subtrees
Token: I            [I]
Token: saw          [I, saw, a, man, with, a, telescope, .]
Token: a            [a]
Token: man          [a, man, with, a, telescope]
Token: with         [with, a, telescope]
Token: a            [a]
Token: telescope    [a, telescope]
Token: .            [.]


## **FUNCTION 3**  -  `is_subtree(sentence, segment)`
### Check if a given list of tokens (segment of a sentence) forms a subtree

The function `is_subtree(sentence, segment)` checks if a segment(ordered list of words) in input forms a subtree of the sentence in input.
It calls `extract_subtrees(sentence)` to extract subtree of each token of the sentence, cycle over all subtrees, search a corrispondence between the segment(list of words) and subtree(list of tokens) checking at every turn if they are equal.
(Here I considered the concept of subtree as complete with the root)

Input:
* sentence ==> a string
* segment ==> list of strings

Output:
* boolean: True if the segment forms a subtree of the sentence, False otherwise

The equality between the segment(list of strings) and the subtree(list of Tokens) is given by the function `is_equal(tokenList, stringList)` that checks the length of the two and if each word of the segment is equal to each text of token, basically if the two lists have the same ordered words.

Input:
* tokenList ==> list of tokens
* stringList ==> list of strings

Output:
* boolean: True if tokenList and stringList have the same ordered words, False otherwise

In [150]:
def is_equal(tokenList, stringList):  #check equality between elements of a list of tokens and a list of strings
  if len(tokenList) != len(stringList):
    return False

  for i in range(len(tokenList)):
    if tokenList[i].text != stringList[i]:
      return False
  
  return True


def is_subtree(sentence, segment): #segment => ordered list of words from a sentence
  subtrees = extract_subtrees(sentence)

  for token, subtree in subtrees:
    if(is_equal(subtree, segment)):
        return True

  return False

In [151]:
input_segment1 = ["a","man","with","a","telescope"]
input_segment2 = ["a","man"]
input_segment3 = ["with","a","telescope"]

print("TEST 1")
print("Sentence:\t{}".format(test_sentence))
print("Segment:\t{}".format(input_segment1))
print(is_subtree(test_sentence, input_segment1))
print("\n")

print("TEST 2")
print("Sentence:\t{}".format(test_sentence))
print("Segment:\t{}".format(input_segment2))
print(is_subtree(test_sentence, input_segment2))
print("\n")

print("TEST 3")
print("Sentence:\t{}".format(test_sentence))
print("Segment:\t{}".format(input_segment3))
print(is_subtree(test_sentence, input_segment3))
print("\n")

TEST 1
Sentence:	I saw a man with a telescope.
Segment:	['a', 'man', 'with', 'a', 'telescope']
True


TEST 2
Sentence:	I saw a man with a telescope.
Segment:	['a', 'man']
False


TEST 3
Sentence:	I saw a man with a telescope.
Segment:	['with', 'a', 'telescope']
True




## **FUNCTION 4**  -  `get_head(span)`
### Identify head of a span, given its tokens

This function given in input a string rappresenting a span(sequence of words),  returns the head of the span.
It simply parses the span and extracts its root, cycling over all token and checking if its dependency relation points to a root(or its head is itself).
In case the span is contains two sentences or maybe a sequence of words which the parser recognize as more sentences, the function return only the first root that it meets.

Input:
* span ==> string

Output:
* string: text of the root of the span


In [ ]:
def get_head(span):     #identify head of a span
  doc = nlp(span)

  return [token for token in doc if token.head == token][0].text   # the condition can be changed with (token.dep_ == 'ROOT')  or  we can use also Span object of spacy


In [ ]:
print("Test 1:")
span1 = "Give it to me! Right now."   #to test return single word 
print("Span:",span1)
print("Head:",get_head(span1))

print("\n")

print("Test 2:")
span2 = "boowling water bad under down mountain"
print("Span:",span2)
print("Head:",get_head(span2))

Test 1:
Span: Give it to me! Right now.
Head: Give


Test 2:
Span: boowling water bad under down mountain
Head: water


## **FUNCTION 5**  -  `extract_spans(sentence)`
### Extract sentence subject, direct object and indirect object spans

This function given in input a string rappresenting a sentence, returns the spans for subject (`nsubj`), direct object (`dobj`) and indirect object (`dative`) if they are present, otherwise nothing.

I used the label `dative` instead of `iobj` because I have printed all the labels in the trained pipeline `en_core_web_sm` with the command `nlp.get_pipe("tagger").labels` and I have noticed that `iobj` is not present.
After some web research I found that that pipeline uses ClearNLP labels where `iobj` is deprecated and has been replaced by `dative`.

Reference: [ClearNLP](https://github.com/clir/clearnlp-guidelines/blob/master/md/specifications/dependency_labels.md)


The function firstly parses the sentence, cycles over all tokens, checks if the dependency of the token is one of the required dependencies(`nsubj`, `dobj`, `dative`), if yes it fills a dict with texts of the elements of the subtree of that token.

Input:
* sentence ==> string

Output:
* dict: dict with a list of strings connected to the respective dependency

The output dict is structured as follows:
{
  'nsubj':['word1', 'word2', 'word3'],
  'dobj':['word1', 'word2', 'word3'],
  'dative':['word1', 'word2', 'word3']
}

In [ ]:
def extract_spans(sentence):
  doc = nlp(sentence)

  span_dict = {
      'nsubj':[],
      'dobj':[],
      'dative':[]
  }

  for token in doc:
    if token.dep_ == 'nsubj' or token.dep_ == 'dobj' or token.dep_ == 'dative':
      for descendant in token.subtree:                    # instead of cycle on elements of the subtree we can use left_edge and right_edge attributes that gives directly first and last element of the token's subtree
        span_dict[token.dep_].append(descendant.text)     #span_dict[token.dep_] = doc[token.left_edge.i : token.right_edge.i+1]
  
  return span_dict


In [ ]:
sentence = "I saw a man with a telescope."

spans = extract_spans(sentence)

print("Sentence:", sentence)
print("Spans:")
for item in spans.items():
  print(item)

Sentence: I saw a man with a telescope.
Spans:
('nsubj', ['I'])
('dobj', ['a', 'man', 'with', 'a', 'telescope'])
('dative', [])
